# Cities2Locations.py

This script converts a list of cities into a list of PYOWM locations. It should be executed when the **cities.csv** file is modified.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import pyowm
from mpl_toolkits.basemap import Basemap, cm

In [ ]:
#Retrieve api-key
with open('../api-key/owm-api-key.txt', 'r') as key_file:
    key = key_file.read()

owm = pyowm.OWM(key)

dic = owm.city_id_registry()

In [ ]:
#Read cities list
cities = pd.read_csv("../data/current-version/cities.csv", ';')

In [ ]:
#Locations lookup in pyowm DB
locrows = []
for index, row in cities.iterrows():
    lookup = dic.locations_for(row['City'].decode("utf-8"), country=row['Country'])
    if lookup==[]:
        print("Cannot find: %s" % row['City'])
    elif len(lookup)>1 and not np.isnan(row['ID']):
        for lu in lookup:
            if lu.get_ID() == row['ID']:
                locrows.append([lu])
                break
    else:
        locrows.append([lookup[0]])

In [ ]:
#Build location dataframe to export
id20s = []
elt_id20 = []
dfrows = locrows

for row in dfrows:
    cit = row[0]
    row.append(cit.get_ID())
    row.append(cit.get_lat())
    row.append(cit.get_lon())
    row[0] = cit.get_name().encode("utf-8")

df = pd.DataFrame(dfrows, columns=['City', 'ID', 'Lat', 'Lon'])

In [ ]:
#Create locations.csv file
df.to_csv('../data/current-version/locations.csv')

### 2) Check the locations found

In [ ]:
#Get cities coordinates
lon = []
lat = []
for city in dfrows:
    lon.append(city[3])
    lat.append(city[2])
    
#Plot the cities on a map
plt.figure(figsize=(24,24))
map = Basemap(llcrnrlon=-10.56,llcrnrlat=34.65,urcrnrlon=34.88,urcrnrlat=74.17, resolution = 'l', epsg=4668)
map.drawmapboundary(fill_color='white')
map.fillcontinents(color='coral',lake_color='white')
map.drawcoastlines()
x, y = map(lon, lat)
map.plot(x, y, 'bo', markersize=5)